In [ ]:
!pip install -qqq -U peft trl datasets bitsandbytes

In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)

from trl import SFTTrainer


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "Akil15/evol_20k_filter"

# Fine-tuned model name
new_model = "Akil15/finetune_llama_v_0.1"


In [3]:
# Bit-sandbytes configuration to downsize large model to small bit size model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# # Check GPU compatibility with bfloat16
# if compute_dtype == torch.float16 and use_4bit:
#     major, _ = torch.cuda.get_device_capability()
#     if major >= 8:
#         print("=" * 80)
#         print("Your GPU supports bfloat16: accelerate training with bf16=True")
#         print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code = True
)

model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [4]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [5]:

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 32

# Dropout probability for LoRA layers
lora_dropout = 0.05

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)


model = get_peft_model(model, peft_config)


In [14]:

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 4

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25



In [15]:
#  Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)


In [8]:
################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [9]:
# List the files in the dataset directory
import os

dataset_path = '//kaggle/input/instruct-set'
files_in_dataset = os.listdir(dataset_path)
print(files_in_dataset)

['converted_alpaca_20k.csv']


In [10]:
import pandas as pd

# Load the CSV file into a Pandas DataFrame
csv_file_path = os.path.join(dataset_path, 'converted_alpaca_20k.csv')
df = pd.read_csv(csv_file_path,low_memory=False)
# dropping unecessary columns
df = df.iloc[:,:3]
df.head()

,input,instruction,output
0,0,Create an array of length 5 which contains all...,"arr = [2, 4, 6, 8, 10]"
1,1,Formulate an equation to calculate the height ...,Height of triangle = opposite side length * si...
2,2,Write a replace method for a string class whic...,"def replace(self, replace_with):\n new_stri..."
3,3,Create an array of length 15 containing number...,"arr = [3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33..."
4,4,Write a function to find the number of distinc...,def find_num_distinct_states(matrix):\n sta...


In [11]:
from datasets import Dataset
# Convert Pandas DataFrame to a datasets.Dataset
custom_dataset = Dataset.from_pandas(df)

In [16]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=custom_dataset,
    peft_config=peft_config,
    dataset_text_field="instruction",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing
)



Map:   0%|          | 0/20013 [00:00<?, ? examples/s]

In [17]:
model.config.use_cache = False
trainer.train()

Step,Training Loss
25,2.747200
50,2.469800
75,1.649900
100,1.964300
125,1.571300
150,1.938500
175,1.499500


KeyboardInterrupt: 

In [ ]:
!huggingface-cli login

In [19]:
import os

output_directory = '/kaggle/working/llama-2-7b-fine_tuned_v_0.1/'
os.makedirs(output_directory, exist_ok=True)


In [20]:
import json


# Convert TrainingArguments to dictionary
training_args_dict = trainer.args.to_dict()

# Save as JSON file
with open("/kaggle/working/llama-2-7b-fine_tuned_v_0.1/training_args.json", "w") as json_file:
    json.dump(training_args_dict, json_file)


In [39]:
training_args_dict

{'output_dir': './results',
 'overwrite_output_dir': False,
 'do_train': False,
 'do_eval': False,
 'do_predict': False,
 'evaluation_strategy': 'no',
 'prediction_loss_only': False,
 'per_device_train_batch_size': 4,
 'per_device_eval_batch_size': 8,
 'per_gpu_train_batch_size': None,
 'per_gpu_eval_batch_size': None,
 'gradient_accumulation_steps': 4,
 'eval_accumulation_steps': None,
 'eval_delay': 0,
 'learning_rate': 0.0002,
 'weight_decay': 0.001,
 'adam_beta1': 0.9,
 'adam_beta2': 0.999,
 'adam_epsilon': 1e-08,
 'max_grad_norm': 0.3,
 'num_train_epochs': 1,
 'max_steps': -1,
 'lr_scheduler_type': 'cosine',
 'lr_scheduler_kwargs': {},
 'warmup_ratio': 0.03,
 'warmup_steps': 0,
 'log_level': 'passive',
 'log_level_replica': 'warning',
 'log_on_each_node': True,
 'logging_dir': './results/runs/Jan27_18-44-05_e9b373ccbedc',
 'logging_strategy': 'steps',
 'logging_first_step': False,
 'logging_steps': 25,
 'logging_nan_inf_filter': True,
 'save_strategy': 'steps',
 'save_steps': 0,
 

In [29]:
trainer.lr_scheduler.state_dict()

{'base_lrs': [0.0002, 0.0002],
 'last_epoch': 199,
 'verbose': False,
 '_step_count': 200,
 '_get_lr_called_within_step': False,
 '_last_lr': [0.00019143163189119916, 0.00019143163189119916],
 'lr_lambdas': [{}, {}]}

In [38]:
model.config

LlamaConfig {
  "_name_or_path": "NousResearch/Llama-2-7b-chat-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false

In [21]:
import torch

# Save optimizer and scheduler states
torch.save(trainer.optimizer.state_dict(), "/kaggle/working/llama-2-7b-fine_tuned_v_0.1/optimizer_state.pth")
torch.save(trainer.lr_scheduler.state_dict(), "/kaggle/working/llama-2-7b-fine_tuned_v_0.1/scheduler_state.pth")
model.save_pretrained("/kaggle/working/llama-2-7b-fine_tuned_v_0.1/")


In [22]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_key_val = user_secrets.get_secret("hf_key")

In [23]:
from huggingface_hub import HfApi
api = HfApi()

In [24]:
api.upload_folder(folder_path = "/kaggle/working/llama-2-7b-fine_tuned_v_0.1/",
                  path_in_repo = ".",
                  repo_id = "Akil15/finetune_llama_v_0.1",
                  repo_type = "model",
                  token =hf_key_val )

scheduler_state.pth:   0%|          | 0.00/639 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

optimizer_state.pth:   0%|          | 0.00/269M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Akil15/finetune_llama_v_0.1/commit/a6728072f0dab09b311bdac934ecb7b5f9ea3363', commit_message='Upload folder using huggingface_hub', commit_description='', oid='a6728072f0dab09b311bdac934ecb7b5f9ea3363', pr_url=None, pr_revision=None, pr_num=None)

In [27]:

tokenizer.push_to_hub(
    "Akil15/finetune_llama_v_0.1", use_auth_token=hf_key_val
)


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Akil15/finetune_llama_v_0.1/commit/3ce60ef6fc7f2a036bc19f66c4f68f8d7164681e', commit_message='Upload tokenizer', commit_description='', oid='3ce60ef6fc7f2a036bc19f66c4f68f8d7164681e', pr_url=None, pr_revision=None, pr_num=None)

In [41]:

model.push_to_hub(
    "Akil15/finetune_llama_v_0.1", use_auth_token=hf_key_val
)


/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py:821: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Akil15/finetune_model_llama_v_0.1/commit/6a4fd97458a19b435dd4d77105fcee336fb292d0', commit_message='Upload model', commit_description='', oid='6a4fd97458a19b435dd4d77105fcee336fb292d0', pr_url=None, pr_revision=None, pr_num=None)